# "2019 February One Day Holding Long Short Portfolio BackTest"
> "2019 February Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [78]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [79]:
#hide_input
#df_tes=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20194dayreturnpred2019train60K.csv')
#df_tes=pd.read_csv('D:\\Downloads\\1day60kvocab20192020testcumulative.csv')

In [80]:
#hide
df_tes['returnpred']=df_tes['labelr1']*df_tes['pred']
df_test['returnpred']=df_test['labelr1']*df_test['pred']

In [81]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20190201].loc[df_tes.Date<20190301].sort_values(by='Date',ascending=True).copy().reset_index(drop=True)
df_long=df_tes.loc[df_tes.pred==1].copy().reset_index(drop=True)
df_short=df_tes.loc[df_tes.pred==-1].copy().reset_index(drop=True)
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [82]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190201].loc[df_test.Date<20190301].sort_values(by='Date',ascending=True).copy().reset_index(drop=True)
df_longa=df_test.loc[df_test.pred==1].copy().reset_index(drop=True)
df_shorta=df_test.loc[df_test.pred==-1].copy().reset_index(drop=True)
#df_testp.reset_index(drop=True, inplace=True)

In [83]:
#hide
df_tes['dailyaveragereturn']=0.0
for date in df_tes['Date'].unique():
    df_tes.loc[df_tes['Date']==date,'dailyaveragereturn']=np.prod(1+(df_tes.loc[df_tes['Date']==date]['returnpred']/100))**(1/len(df_tes.loc[df_tes['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

In [84]:
#hide
df_test['dailyaveragereturn']=0.0
for date in df_test['Date'].unique():
    df_test.loc[df_test['Date']==date,'dailyaveragereturn']=np.prod(1+(df_test.loc[df_test['Date']==date]['returnpred']/100))**(1/len(df_test.loc[df_test['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

In [85]:
#hide
df_a=df_tes.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_test.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

In [86]:
#hide
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])

df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

In [87]:
#hide
df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

**Long-Short Portfolio**

In [88]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [89]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2019-02-01,1.000529,0.996275,1.006776,1.007001
1,2019-02-04,1.020219,0.984513,1.011517,1.013870
2,2019-02-05,1.012718,0.982596,1.009267,1.013023
3,2019-02-06,1.015864,0.973536,0.999823,1.004215
4,2019-02-07,1.014529,0.966931,1.000499,1.001693
5,2019-02-08,1.011946,0.949488,1.001208,0.999570
6,2019-02-11,1.022921,0.951703,1.014114,1.014438
7,2019-02-12,1.006781,0.954150,1.017181,1.019126
8,2019-02-13,1.029616,0.963048,1.014483,1.014982
9,2019-02-14,0.986247,0.961524,1.025520,1.032691


In [90]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)

**Long portfolio**

In [91]:
#hide
df_long['dailyaveragereturn']=0.0
for date in df_long['Date'].unique():
    df_long.loc[df_long['Date']==date,'dailyaveragereturn']=np.prod(1+(df_long.loc[df_long['Date']==date]['returnpred']/100))**(1/len(df_long.loc[df_long['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_longa['dailyaveragereturn']=0.0
for date in df_longa['Date'].unique():
    df_longa.loc[df_longa['Date']==date,'dailyaveragereturn']=np.prod(1+(df_longa.loc[df_longa['Date']==date]['returnpred']/100))**(1/len(df_longa.loc[df_longa['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_a=df_long.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_longa.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])
    
df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

In [92]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [93]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2019-02-01,1.032504,1.016543,1.006776,1.007001
1,2019-02-05,1.032504,1.013897,1.009267,1.013023
2,2019-02-06,1.023328,0.977920,0.999823,1.004215
3,2019-02-08,1.030356,1.005629,1.001208,0.999570
4,2019-02-11,1.020989,1.008220,1.001208,0.999570


In [94]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)

**Short Portfolio**

In [95]:
#hide
df_short['dailyaveragereturn']=0.0
for date in df_short['Date'].unique():
    df_short.loc[df_short['Date']==date,'dailyaveragereturn']=np.prod(1+(df_short.loc[df_short['Date']==date]['returnpred']/100))**(1/len(df_short.loc[df_short['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_shorta['dailyaveragereturn']=0.0
for date in df_shorta['Date'].unique():
    df_shorta.loc[df_shorta['Date']==date,'dailyaveragereturn']=np.prod(1+(df_shorta.loc[df_shorta['Date']==date]['returnpred']/100))**(1/len(df_shorta.loc[df_shorta['Date']==date]))
    #print(np.prod(1+(df_tes.loc[df_tes['Date']==date]['labelr1']/100))**(1/len(df_tes.loc[df_tes['Date']==date])))

df_a=df_short.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)
df_b=df_shorta.drop_duplicates(subset = ["Date"],keep='last').reset_index(drop=True)

df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a.loc[l,'cumreturn1b']=np.prod(df_a['dailyaveragereturn'][:l+1])
    
df_b['cumreturn1c']=0.0
for l in range(len(df_b)):
    df_b.loc[l,'cumreturn1c']=np.prod(df_b['dailyaveragereturn'][:l+1])

df_a=pd.merge(df_a,df_b[['Date','cumreturn1c']], right_on='Date', left_on='Date',how='inner')

In [96]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [97]:
#hide_input
df_a[['Date','cumreturn1b','cumreturn1c','cumreturndow','cumreturnsp500']]

,Date,cumreturn1b,cumreturn1c,cumreturndow,cumreturnsp500
0,2019-02-01,0.993559,0.988694,1.006776,1.007001
1,2019-02-04,1.013111,0.974114,1.011517,1.013870
2,2019-02-05,1.004989,0.967727,1.009267,1.013023
3,2019-02-06,1.009844,0.980385,0.999823,1.004215
4,2019-02-07,1.008517,0.967285,1.000499,1.001693
5,2019-02-08,1.004769,0.905746,1.001208,0.999570
6,2019-02-11,1.019351,0.907721,1.014114,1.014438
7,2019-02-12,1.003268,0.916297,1.017181,1.019126
8,2019-02-13,1.026023,0.928737,1.014483,1.014982
9,2019-02-14,0.982806,0.919569,1.025520,1.032691


In [98]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs 8KPortfolio (brown) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_a).mark_trail(color='brown').encode(
    x='Date:T',
    y='cumreturn1c:Q',
    size='cumreturn1c',

).properties(
    width=1000,
    height=1000
)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3 + a4# + a5

alt.LayerChart(...)